In [1]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
# Weights
w_c, w_l, w_s = sp.symbols('w_c w_l w_s')
# Token balances
x_c, x_l, x_s = sp.symbols('x_c x_l x_s')
# Prices - in units of locked up collateral
v = sp.symbols('v') 
# Decimal scaling 
d_c, d_p = sp.symbols('d_c d_p')
# Collateral backing L + S 
# e.g. for $100.000_000_000_000_000_000 = 1.000_000 mt
# this is 100*10**18 / 1*10**6 = 10**14
C = sp.symbols('C')

In [4]:
sol = sp.solve(
    [w_c + w_l + w_s - 1,   # Weights sum to 1
     x_c/w_c - x_l/w_l*v*C, # Long token price
     x_c/w_c - x_s/w_s*(1-v)*C  # Short token price
    ]
     , [w_c, w_l, w_s])

In [5]:
sol[w_c]

x_c/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [6]:
sol[w_l]

C*v*x_l/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [7]:
sol[w_s]

-C*x_s*(v - 1)/(C*v*x_l - C*x_s*(v - 1) + x_c)

In [8]:
sol

{w_c: x_c/(C*v*x_l - C*x_s*(v - 1) + x_c),
 w_l: C*v*x_l/(C*v*x_l - C*x_s*(v - 1) + x_c),
 w_s: -C*x_s*(v - 1)/(C*v*x_l - C*x_s*(v - 1) + x_c)}

Consider now pair balance and difference with $x_l = (x_p + x_d)/2$ and $x_s = (x_p - x_d)/2$ so that we can introduce a budget constraint $x_c + C*x_p = b_c$

In [10]:
x_p, x_d = sp.symbols('x_p x_c')
subs_dict = {x_l: (x_p + x_d)/2, x_s: (x_p - x_d)/2}

In [20]:
b_c = sp.symbols('b_c')
budget_subs_dict = {x_c: b_c - C*x_p}

In [13]:
sol[w_c].subs(subs_dict).simplify()

2*x_c/(2*C*v*x_c - C*x_c + C*x_p + 2*x_c)

In [14]:
sol[w_l].subs(subs_dict).simplify()

C*v*(x_c + x_p)/(2*C*v*x_c - C*x_c + C*x_p + 2*x_c)

In [18]:
sol[w_s].subs(subs_dict).simplify()

C*(v - 1)*(x_c - x_p)/(C*v*(x_c + x_p) + C*(v - 1)*(x_c - x_p) + 2*x_c)

In [19]:
(sol[w_c] + sol[w_l] + sol[w_s]).simplify()

1

In [29]:
sol[w_c].subs(subs_dict).subs(budget_subs_dict).simplify()

2*(-C*x_p + b_c)/(C*v*(-C*x_p + b_c + x_p) - 2*C*x_p - C*(v - 1)*(C*x_p - b_c + x_p) + 2*b_c)

In [26]:
sol[w_c].subs(subs_dict).subs(budget_subs_dict).simplify().diff(x_p).simplify()

-2*C*b_c/(2*C**2*v*x_p - C**2*x_p - 2*C*b_c*v + C*b_c + C*x_p - 2*b_c)**2